In [ ]:
import pandas as pd
import cPickle as pickle
from IPython.display import display
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# from matplotlib import rc
# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
# rc('text', usetex=True)

%matplotlib inline

In [ ]:
remap_dict = {
    ('unmeasure', 'blur'): 'unmeasure-blur',
    ('unmeasure', 'inpaint-tv') : 'unmeasure-inpaint-tv',
    ('baseline', 'None') : 'ignore',
    ('ambient', 'None') : 'AmbientGAN (ours)',
    ('unmeasure', 'wiener'): 'unmeasure-weiner',
    'pad_rotate_project': 'Pad-Rotate-Project',
    'pad_rotate_project_with_theta': r'Pad-Rotate-Project-$\theta$',
}

In [ ]:
def save_plot(save_path):
    pdf = PdfPages(save_path)
    pdf.savefig(bbox_inches='tight')
    pdf.close()   
    
def errorbar(x, y, std):
    (_, caps, _) = plt.errorbar(x, y, yerr=1.96*std,
                                marker='o', markersize=5, capsize=5,
                                alpha=0.8, linewidth=2)
    for cap in caps:
        cap.set_markeredgewidth(1)

def plot(plot_spec, df):
    groups = df.groupby(plot_spec['group_by'])
    legends = []
    x_var = plot_spec['x_var']
    y_var = plot_spec['y_var']
    std_var = plot_spec['std_var']
    
    plt.figure(figsize=[6, 4])
    
    for key in sorted(groups.groups):
        val = groups.groups[key]
        group_df = df.loc[val, :]
        cols = [x_var, y_var, std_var]
        group_df.sort_values(by=[x_var], inplace=True)
        x = np.array(group_df[x_var])
        y = np.array(group_df[y_var])
        if std_var is not None:
            std = np.array(group_df[std_var])
            errorbar(x, y, std)
        else:
            plt.plot(x, y, '-o', alpha=0.8, linewidth=2)
        legend = remap_dict[key]
        legends.append(legend)
        
    ## Prettify
    # axis
    plt.gca().set_ylim(bottom=0)
    # plt.gca().set_xscale("log", nonposx='clip')
    # plt.gca().set_xlim([-0.1, 1.1])

    # labels, ticks, titles
    # ticks = [10, 25, 50, 100, 200, 300, 400, 500, 750]
    # labels = [10, 25, 50, 100, 200, 300, 400, 500, 750]
    # plt.xticks(ticks, labels, rotation=90)
    plt.xlabel(plot_spec['x_label'], fontsize=14)
    plt.ylabel(plot_spec['y_label'], fontsize=14)

    # Legends
    plt.legend(legends, fontsize=12.5, loc=3)
    # plt.ylim([0, 0.1])
    
    plt.title(plot_spec['title'])

In [ ]:
def convert_df(df, convert_spec):
    for colname in convert_spec['float']:
        df.loc[:, colname] = df[colname].apply(np.float32)
    return df

def filter_df(df, filter_spec):
    df = df[filter_spec['cols']]
    filter_ = True
    for colname, values in filter_spec['in'].iteritems():
        filter_ = filter_ & df[colname].isin(values)
    df = df[filter_]
    return df

In [ ]:
def main(df_filepath, convert_spec, filter_spec, plot_spec):
    with open(df_filepath, 'rb') as df_pkl_file:
        df = pickle.load(df_pkl_file)
    df = convert_df(df, convert_spec)
    df = filter_df(df, filter_spec)    
    display(df)
    plot(plot_spec, df)

In [ ]:
df_filepath = './results/df_hparams_metrics.pkl'

convert_spec = {
    'float': [
        'drop_prob',
        'm_inception_score_mean', 
#         'm_inception_score_std',
    ]        
}

filter_spec = {
    'cols' : ['measurement_type', 
              'train_mode', 
              'drop_prob',
              'unmeasure_type', 
              'm_inception_score_mean',
#               'm_inception_score_std',
              'model_type',
             ],
    'in': {
        'measurement_type': ['drop_independent'],
        'model_type' : ['wgangp'],
    }
}
plot_spec = {
    'group_by': ['train_mode', 'unmeasure_type'],
    # change these to lambda if needed
    'x_var': 'drop_prob',
    'y_var': 'm_inception_score_mean',
#     'std_var': 'm_inception_score_std',
    'std_var': None,
    'x_label': 'Block probability (p)',
    'y_label': 'Inception score',
    'title': 'MNIST, Block-Pixels',
}

main(df_filepath, convert_spec, filter_spec, plot_spec)
fig_savepath = './results/plots/mnist_drop_independent.pdf'
save_plot(fig_savepath)

In [ ]:
df_filepath = './results/df_hparams_metrics.pkl'

convert_spec = {
    'float': ['additive_noise_std', 'm_inception_score_mean']        
}

filter_spec = {
    'cols' : ['measurement_type', 
              'train_mode', 
              'additive_noise_std',
              'unmeasure_type', 
              'm_inception_score_mean'
             ],
    'in': {
        'measurement_type': ['blur_addnoise']
    }
}
plot_spec = {
    'group_by': ['train_mode', 'unmeasure_type'],
    # change these to lambda if needed
    'x_var': 'additive_noise_std',
    'y_var': 'm_inception_score_mean',
    'std_var': None,
    'x_label': 'Noise standard deviation ($\sigma$)',
    'y_label': 'Inception score',
    'title': 'placeholder',
}

main(df_filepath, convert_spec, filter_spec, plot_spec)
fig_savepath = './results/plots/mnist_blur_noise.pdf'
save_plot(fig_savepath)

In [ ]:
df_filepath = './results/df_hparams_metrics.pkl'

convert_spec = {
    'float': ['additive_noise_std', 'm_inception_score_mean']        
}

filter_spec = {
    'cols' : ['measurement_type', 
              'train_mode', 
              'additive_noise_std',
              'unmeasure_type', 
              'm_inception_score_mean',
              'num_angles',
             ],
    'in': {
        'measurement_type': [
            'pad_rotate_project',
            'pad_rotate_project_with_theta',
        ]
    }
}
plot_spec = {
    'group_by': ['measurement_type'],
    # change these to lambda if needed
    'x_var': 'num_angles',
    'y_var': 'm_inception_score_mean',
    'std_var': None,
    'x_label': 'Number of angles',
    'y_label': 'Inception score',
    'title': 'placeholder',
}

main(df_filepath, convert_spec, filter_spec, plot_spec)
plt.axhline(y=8.994174, color='red')
plt.ylim([0, 10])

fig_savepath = './results/plots/mnist_1d.pdf'
save_plot(fig_savepath)